# Práctica 6: PIV

Pipeline: carga de pares A/B, preprocesado (fondo, ecualización), PIV multipase, outliers, derivados.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import cv2
import openpiv.tools as tools
import openpiv.process as process
import openpiv.validation as validation
import openpiv.filters as filters
import openpiv.scaling as scaling

In [ ]:
DATA_DIR = Path('..') / 'data' / 'pairs'
pairs = sorted(DATA_DIR.glob('*.tif'))
pairs[:4]  # espera A1,B1,A2,B2...

In [ ]:
# Ejemplo simple con la primera pareja
frame_a = tools.imread(pairs[0])
frame_b = tools.imread(pairs[1])
u, v, sig2noise = process.extended_search_area_piv(
    frame_a, frame_b, window_size=32, overlap=16, dt=0.01, search_area_size=64
)
x, y = process.get_coordinates(image_size=frame_a.shape, window_size=32, overlap=16)
u, v, mask = validation.sig2noise_val(u, v, sig2noise, threshold = 1.3)
u, v = filters.replace_outliers(u, v, method='localmean', max_iter=3, kernel_size=2)
x, y, u, v = scaling.uniform(x, y, u, v, scaling_factor=1.0)
plt.figure(figsize=(6,6)); plt.quiver(x, y, u, v); plt.gca().invert_yaxis(); plt.title('Campo PIV'); plt.show()